In [1]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [2]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

### GPT questions for assessing task stats

In [3]:
from edsl.questions import QuestionNumerical

def get_tasks_stas(occupation, tasks):
   scenarios = [Scenario({"occupation": occupation, "all_tasks": tasks, "task": task}) for task in tasks]

   q_human_cost = QuestionNumerical(
      question_name = "human_cost", # parameter M in model
      question_text = dedent("""
                           Consider {{ occupation }} as an occupation. 
                           And consider this task: {{ task }}.
                           How long (in minutes) does it take a person to complete this task? 
                           (5 = 5 minutes or less, 480 = a full day, or 8 hours)
                           """),
      min_value = 5, # 5 minutes or less
      max_value = 480, # a full day, or 8 hours
   )

   q_management_cost = QuestionNumerical(
      question_name = "management_cost", # parameter A in model
      question_text = dedent("""
                           Consider {{ occupation }} as an occupation. 
                           And consider this task: {{ task }}.
                           Suppose a human worker "A" is assigned to and completes this task.
                           Now imagine a general-purpose artificial intelligence machine is deployed
                              to automate management process of this task and to validate worker "A"'s output.
                           How long will it take the machine to confirm if worker "A" has completed the task successfully?
                           (5 = 5 minute or less, 480 = a full day, or 8 hours)
                           """),
      min_value = 5, # 5 minutes or less
      max_value = 480, # a full day, or 8 hours
   )

   q_management_difficulty = QuestionNumerical(
      question_name = "management_difficulty", # parameter D in model for version with A
      question_text = dedent("""
                           Consider {{ occupation }} as an occupation.
                           In order to successfully complete the job, a worker needs to do the following tasks: {{ all_tasks }}.
                           Now consider this task in particular: {{ task }}.
                           Suppose the task is managed using a general-purpose artificial intelligence (AI) machine.
                           Given that the machine is not specifically trained in managing tasks of this occupation,
                           how difficult is it that it successfully assesses completion of this task in an attempt?
                           Give an answer that represents the relative difficulty of assessing completion of this task compared to the rest of tasks in occupation.
                           The answer must be a value that has up to two significant digits after the decimal point.
                           (0 = not difficult at all AI can successfully do it in every try, 1 = so difficult AI cannot do it in any attempt)
                           """),
      min_value = 0, # not difficult at all AI can successfully do it in every try
      max_value = 1, # so difficult AI cannot do it in any attempt
   )

#    q_machine_cost = QuestionNumerical(
#       question_name = "machine_cost", # parameter C in model
#       question_text = dedent("""
#                            Consider {{ occupation }} as an occupation. 
#                            In order to successfully complete the job, the following tasks must be successfully done: {{ all_tasks }}.
#                            Suppose a number of human workers are assigned to and complete these tasks.
#                            Now imagine a general-purpose artificial intelligence is deployed to automate all tasks.
#                            How many compute units should the machine spend on average on each tasks to complete the job successfully?
#                              Your answer should not be "None" or "0".
#                            A compute unit can be, for example, the compute power of a single CPU core running for one minute, or the number of API calls to a cloud service.
#                            (1 = 1 compute unit or less, 480 = 480 CPUs working for a minute or 1 CPU working for 480 minutes (equivalent of a full working day), or 480 API calls)
#                            """),
#       min_value = 1, # 1 compute units
#       max_value = 480, # 480 cpus working for a minute, or 480 API calls
#    )

   q_completion_difficulty = QuestionNumerical(
      question_name = "completion_difficulty", # parameter D in model for version with C
      question_text = dedent("""
                           Consider {{ occupation }} as an occupation.
                           In order to successfully complete the job, a worker needs to do the following tasks: {{ all_tasks }}.
                           Now consider this task in particular: {{ task }}.
                           Suppose the task is completed using a general-purpose artificial intelligence (AI) machine.
                           Given that the machine is not specifically trained in managing tasks of this occupation,
                           how difficult is it that it successfully completes this task in an attempt?
                           Give an answer that represents the relative difficulty of completing this task compared to the rest of tasks in occupation.
                           The answer must be a value that has up to two significant digits after the decimal point.
                           (0 = not difficult at all AI can successfully do it in a single try, 1 = so difficult AI cannot do it in any attempt)
                           """),
      min_value = 0, # not difficult at all AI can successfully do it in a single try
      max_value = 1, # so difficult AI cannot do it in any attempt
   )

   # Run questions
   results_human_cost = q_human_cost.by(m4).by(scenarios).run()
   results_management_cost = q_management_cost.by(m4).by(scenarios).run()
   results_management_difficulty = q_management_difficulty.by(m4).by(scenarios).run()
   # results_machine_cost = q_machine_cost.by(m4).by(scenarios).run()
   results_completion_difficulty = q_completion_difficulty.by(m4).by(scenarios).run()

   # Convert outputs to pandas dataframe
   human_cost_df = results_human_cost.to_pandas()
   management_cost_df = results_management_cost.to_pandas()
   management_difficulty_df = results_management_difficulty.to_pandas()
   # machine_cost_df = results_machine_cost.to_pandas()
   completion_difficulty_df = results_completion_difficulty.to_pandas()

   # Subset dataframe
   human_cost_df = human_cost_df[['scenario.task', 'answer.human_cost']]
   management_cost_df = management_cost_df[['scenario.task', 'answer.management_cost']]
   management_difficulty_df = management_difficulty_df[['scenario.task', 'answer.management_difficulty']]
   # machine_cost_df = machine_cost_df[['scenario.task', 'answer.machine_cost']]
   completion_difficulty_df = completion_difficulty_df[['scenario.task', 'answer.completion_difficulty']]

   # Merge outputs
   tasks_stats = pd.merge(human_cost_df, management_cost_df, on='scenario.task', how='inner')
   tasks_stats = pd.merge(tasks_stats, management_difficulty_df, on='scenario.task', how='inner')
   # tasks_stats = pd.merge(tasks_stats, machine_cost_df, on='scenario.task', how='inner')
   tasks_stats = pd.merge(tasks_stats, completion_difficulty_df, on='scenario.task', how='inner')

   # Rename columns
   tasks_stats.rename(columns={'scenario.task': 'task', 
                               'answer.human_cost': 'human_cost', 
                               'answer.management_cost': 'management_cost', 
                               'answer.management_difficulty': 'management_difficulty',
                               # 'answer.machine_cost': 'machine_cost', 
                               'answer.completion_difficulty': 'completion_difficulty'}, inplace=True)
   
   return tasks_stats

In [4]:
def get_tasks(onet_data_path,
              occupation_code):

    # Load the data
    onet = pd.read_csv(onet_data_path)
    onet = onet.sort_values(by=['year', 'occ_code', 'occ_title', 'task_id'])
    onet = onet[onet['year'] == 2023].reset_index(drop=True)

    # Get list of tasks
    my_df = onet[(onet.occ_code == f'{occupation_code}') & (onet.year == 2023)]
    tasks = my_df['task'].unique().tolist()
    tasks = [task.replace("'", "") for task in tasks] # remove apastrophes so partitions DAG code can handle task strings
    return tasks

In [5]:
def get_AI_completion_cost(occupation, tasks):
  scenarios = [Scenario({"occupation": occupation, "all_tasks": tasks})]

  q_machine_cost = QuestionNumerical(
  question_name = "machine_cost", # parameter C in model
  question_text = dedent("""
                        Consider {{ occupation }} as an occupation. 
                        In order to successfully complete the job, the following tasks must be successfully done: {{ all_tasks }}.
                        Suppose a number of human workers are assigned to and complete these tasks.
                        Now imagine a general-purpose artificial intelligence is deployed to automate all tasks.
                        How many compute units should the machine spend on average on each tasks to complete the job successfully?
                          Your answer should not be "None" or "0".
                        A compute unit can be, for example, the compute power of a single CPU core running for one minute.
                        (1 = 1 compute unit or less, 480 = 480 CPUs working for a minute or 1 CPU working for 480 minutes, or a full working day)
                        """),
  min_value = 1, # 1 compute units
  max_value = 480, # 480 cpus working for a minute, or 480 API calls
  )

  # q_machine_cost = QuestionNumerical(
  # question_name = "machine_cost", # parameter C in model
  # question_text = dedent("""
  #                       Consider {{ occupation }} as an occupation. 
  #                       In order to successfully complete the job, the following tasks must be successfully done: {{ all_tasks }}.
  #                       Suppose a number of human workers are assigned to and complete these tasks.
  #                       Now imagine a general-purpose artificial intelligence is deployed to automate all tasks.
  #                       How many compute units should the machine spend on average on each tasks to complete the job successfully?
  #                         Your answer should not be "None" or "0".
  #                       A compute unit can be, for example, the number of API calls to a cloud service.
  #                       (1 = 1 compute unit or less, 10000 = 10000 API calls)
  #                       """),
  # min_value = 1, # 1 compute units
  # max_value = 10000, # 480 cpus working for a minute, or 480 API calls
  # )

  # Run question
  results_machine_cost = q_machine_cost.by(m4).by(scenarios).run()

  # Convert outputs to pandas dataframe
  machine_cost_df = results_machine_cost.to_pandas()

  return machine_cost_df['answer.machine_cost'].iloc[0]

### Fetch Occupation Task Stats

In [6]:
onet_data_path = f'{data_path}/data/onet_occupations_yearly.csv'

occupation_list = ['pileDriverOperators', 'dredgeOperators', 'gradersAndSortersForAgriculturalProducts',
                   'insuranceUnderwriters', 'insuranceAppraisersForAutoDamage', 'floorSandersAndFinishers', 
                   'reinforcingIronAndRebarWorkers', 'travelAgents', 'dataEntryKeyer', 
                   'athletesAndSportsCompetitors', 'audiovisualEquipmentInstallerAndRepairers', 'hearingAidSpecialists', 
                   'personalCareAides', 'proofreadersAndCopyMarkers', 'chiropractors', 
                   'shippingReceivingAndInventoryClerks', 'cooksShortOrder', 'orthodontists',
                   'subwayAndStreetcarOperators', 'packersAndPackagersHand', 'hoistAndWinchOperators', 
                   'forgingMachineSettersOperatorsAndTenders', 'avionicsTechnicians', 'dishwashers', 
                   'dispatchersExceptPoliceFireAndAmbulance', 'familyMedicinePhysicians', 'MachineFeedersAndOffbearers'
                   ]


# occupation_list = ['travelAgents']

occupation_list = ['sociologists', 'dancers', 'carpetInstallers']

In [7]:
import time
start_time = time.time()

for occupation in occupation_list:
    occupation_start_time = time.time()
    print(f'--------------- Running: {occupation} ---------------')

    # Generate occupation-specific strings
    GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

    # Get occupation tasks
    tasks = get_tasks(onet_data_path, occupation_code)
    print(f'Number of tasks: {len(tasks)}')

    # Ask GPT to assess tasks' stats
    tasks_stats = get_tasks_stas(GPT_input_occupation, tasks)

    # Ask GPT to assess AI completion cost for occupation
    machine_cost = get_AI_completion_cost(GPT_input_occupation, tasks)

    # add AI completion cost to tasks stats and change order of columns
    tasks_stats['machine_cost'] = machine_cost
    tasks_stats = tasks_stats[['task', 'human_cost', 'management_cost', 'management_difficulty', 'machine_cost', 'completion_difficulty']]


    # add "Target" task to tasks list
    aux_df = pd.DataFrame({'task': ['"Target"'],
                        'human_cost': [0],
                        'management_cost': [0],
                        'management_difficulty': [0], 
                        'machine_cost': [0],
                        'completion_difficulty': [0],})
    tasks_stats = pd.concat([tasks_stats, aux_df], ignore_index=True)

    # Save output
    tasks_stats.to_csv(f'{occupation_folder}/{occupation}_taskStats.csv', index=False)

    occupation_end_time = time.time()
    occupation_execution_time = occupation_end_time - occupation_start_time
    print(f"**** {occupation} runtime: {occupation_execution_time:.2f} seconds ****\n")

end_time = time.time()
execution_time = (end_time - start_time)/60
print(f"\n\nTotal Runtime: {execution_time:.2f} minutes")

--------------- Running: sociologists ---------------
Number of tasks: 16
**** sociologists runtime: 7.76 seconds ****

--------------- Running: dancers ---------------
Number of tasks: 14
**** dancers runtime: 15.05 seconds ****

--------------- Running: carpetInstallers ---------------
Number of tasks: 15
**** carpetInstallers runtime: 6.84 seconds ****



Total Runtime: 0.49 minutes
